In [7]:
import ollama
import pandas as pd

import json
import datetime
from functools import reduce
from utils import is_in_colab
from base import persian_text_preprocessing

In [ ]:
comments_file_path = './data/incredible_offers_product_comments_20250622_120006.json'
with open(comments_file_path, "r") as f:
    product_comments_map = json.load(f)

In [ ]:
system_prompt = '''# Persian E-commerce Sentiment Classifier

**Task:** Classify Persian product reviews as 0 (Negative), 1 (Positive), or 2 (Neutral).

## Classification Process:

1. **Read the review carefully**
2. **Identify key sentiment indicators**
3. **Apply classification rules in order**
4. **Output format: [digit] - [brief reasoning]**

## Classification Rules (in priority order):

### Priority 1: Explicit Recommendations/Anti-recommendations
- **Positive (1):** "پیشنهاد میکنم", "حتما بخرید", "چشم بسته بخرید"
- **Negative (0):** "پیشنهاد نمیکنم", "نخرید", "صرف نظر کنید"

### Priority 2: Overall Satisfaction Statements
- **Positive (1):** "راضیم", "از خریدم راضی هستم", "دوباره میخرم"
- **Negative (0):** "پشیمونم", "ناراضی", "از خریدم پشیمان شدم"

### Priority 3: Mixed Sentiment Resolution
- Look for "ولی" (but) - final conclusion after "but" typically dominates
- "در کل" (overall) statements override specific complaints/praises
- Value-based acceptance: "معمولی ولی برای این قیمت خوب" → Positive

### Priority 4: Dominant Sentiment
- **Positive indicators:** راضیم، عالی، خوب، کاربردی، مناسب، با کیفیت
- **Negative indicators:** بد، خراب، مشکل، کار نمیکنه، افتضاح، آشغال
- **Count positive vs negative indicators**

### Priority 5: Neutral Classification
- Only when NO clear positive/negative indicators present
- Pure factual statements: specifications, delivery info, descriptions
- Balanced pros/cons with no preference indicated

## Few-Shot Examples:

**Example 1:**
Input: "بعضیا نوشته بودن وزن گوشی رو نمیتونه تحمل کنه ولی من مشکلی نداشتم"
Output: 1 - Defending product against others' criticisms implies positive experience

**Example 2:**
Input: "کار میکنه ولی کیفیت خیلی پایینه. برای این قیمت انتظار بیشتری داشتم"
Output: 0 - Functionality acknowledged but quality disappointment + unmet expectations

**Example 3:**
Input: "خوبه برای بابام ابی سفارش دادم ولی اون آبی که تو عکسه نیست. اگه همین رنگ رو میذاشتن آبی انتخاب نمیکردم"
Output: 0 - Says "خوبه" but wrong color caused regret about purchase decision

**Example 4:**
Input: "بستگی به استفاده داره. برای بعضیا مناسبه برای بعضیا نه"
Output: 2 - Conditional statement without personal opinion or preference

**Example 5:**
Input: "جنس چینی ولی محکم ساخته شده. تا الان مشکلی نداشتم"
Output: 1 - "چینی" is factual, positive experience with build quality and reliability

## Edge Cases:
- **Sarcasm:** Look for context clues, treat as opposite sentiment
- **Comparative reviews:** Focus on final preference/recommendation
- **Uncertainty:** If genuinely ambiguous, default to 2 (Neutral)

## Output Format:
[0/1/2] - [Brief reasoning in English]

Process the review step by step and provide your classification.
'''

label_map = {
    0: "Negative",
    1: "Positive",
    2: "Neutral"
}

In [22]:
comments = reduce(lambda acc, arr: acc + arr, product_comments_map.values(), [])
filtered_comments = list(filter(lambda c: c['is_buyer'] == 1 and len(c['body'].strip().split()) >= 5, comments))
filtered_comments = list(map(lambda c: persian_text_preprocessing(c['body']), filtered_comments))

In [23]:
filtered_comments[:5]

['ایرفون کوچیک متناسب با ارگونومی گوش پد های گوششم میتونید متناسب با سایز گوشتون عوض کنید تا صداهای مزاحم خارجی به کمترین حد خودشون برسن, صدای کار شده تمیز با بیس عمیق که لذت موسیقی خالص رو بهتون میده. با توجه به مبلغی که هزینه میکنید, محصول خوبی میگیرید.',
 'کیفیت عالی پخش. بعد از ده روز استفاده; مشخصات کالا درست بود, بیس سه بعدی , سرعت شارژ , زمان نگهداری شارژ و ظرافت و زیبایی ,, بسته بندی و زمان رسید کالا هم دقیق بود',
 'کیفیت صدای خوبی داره باکس شارژش هم خیلی کاربردیه هم گوشی های هنزفری رو شارژ میکنه و هم میشه در مواقع اظطراری گوشی موبایلتون رو باهاش شارژ کنید بنظرم تخفیفی که گذاشتن رو از دست ندید',
 'جنس بدنه ضعیف یک کالا با دو قیمت متفاوت با همین فروشنده قرار داشت کیفت صدا خوبه ولی برای مکالمه صدا خوب شنیده میشه ولی صدای خودم با هدفون اصلا واضح نبود ولی خیلی زود به دستم رسید و سالم بود',
 'ظاهر جذاب و زیبا در عین حال ساده کیفیت صدا واقعا خوبه بیس خوبی داره باخیال راحت خرید کنید']

In [ ]:
if not is_in_colab():
    model_name = "gemma3:4b"

    def label_comment_sentiment(system_prmopt, user_message, model="gemma3:4b"):
        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": system_prmopt},
                {"role": "user", "content": user_message},
            ]
        )

        return response['message']['content']

    labels = []
    reasons = []
    for i, comment in enumerate(filtered_comments):
        response: str = label_comment_sentiment(system_prompt, f'user comment: {comment}', model_name)
        label = int(response.split('-')[0].strip())
        reason = response.split('-')[1].strip() 
        reasons.append(reason)
        labels.append(label)
        if i % 100 ==0:
            print(f"comment_{i}: {comment}")
            print(f"Sentiment: {str(label) + ' (' +label_map[label] + ')'} Model Reason: {reason}\n")

comment_0: ایرفون کوچیک متناسب با ارگونومی گوش پد های گوششم میتونید متناسب با سایز گوشتون عوض کنید تا صداهای مزاحم خارجی به کمترین حد خودشون برسن, صدای کار شده تمیز با بیس عمیق که لذت موسیقی خالص رو بهتون میده. با توجه به مبلغی که هزینه میکنید, محصول خوبی میگیرید.
Sentiment: 1 (Positive) Model Reason: Positive: Mentions "clean sound with deep bass" and "good product for the price," indicating satisfaction.

comment_100: خیلی به کارتون میاد بخرید
Sentiment: 1 (Positive) Model Reason: “خیلی به کارتون میاد” translates to “This would be very useful for you,” indicating a positive recommendation.

comment_200: عالیه واقعا چیزی بود که دنبالش بودم جنسش محکمه و اینکه درجه باز شدنش قابل تنظیم هست خیلی عالیه و یه چیزی که خیلی خوبه اینکه موبایل روش سنگینی نمیکنه برگرده به عقب و بیفته. اندازش نسبت به کف دست رو میتونین توی عکس که گذاشتم ببینین.
Sentiment: 1 (Positive) Model Reason: Positive: The review highlights several positive aspects – durable material, adjustable opening, and doesn't cause the

In [ ]:
if is_in_colab():
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    from transformers.pipelines import pipeline

    model_name = "HooshvareLab/bert-fa-base-uncased-sentiment-digikala"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    classifier = pipeline("text-classification",
                        model=model, 
                        tokenizer=tokenizer) 
    
    hoosvare_predictions: list[dict] = classifier.predict(filtered_comments) # type: ignore
    label_map = {
        'not_recommended': 0,
        'recommended': 1,
        'no_idea': 2
    }
    hoosvare_predictions = list(map(lambda entry: {**entry, 'label': label_map[entry['label']]}, hoosvare_predictions))
    hooshvare_df = pd.DataFrame(hoosvare_predictions, index=None)
    hooshvare_df['text'] = filtered_comments
    hooshvare_df.to_csv("./data/bert_fa_base_uncased_sentiment_digikala_predictions.csv", index=False)

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
df = pd.DataFrame({"text": filtered_comments, 'label': labels, 'reason': reasons})
df.to_csv(f'./data/{model_name.replace(':','_')}_filtered_comments_labels_{timestamp}.csv', index=False)

In [ ]:
ollama_df = pd.read_csv('./data/gemma3_4b_filtered_comments_labels_20250703_152906.csv', index_col=None)

In [15]:
ollama_df['label'].value_counts()

label
1    1673
0     725
2      97
Name: count, dtype: int64

In [ ]:
hooshvare_df = pd.read_csv('./data/bert_fa_base_uncased_sentiment_digikala_predictions.csv', index_col=False)
hooshvare_df.head()

,label,score,text
0,1,0.937479,ایرفون کوچیک متناسب با ارگونومی گوش پد های گوش...
1,1,0.981608,کیفیت عالی پخش. بعد از ده روز استفاده; مشخصات ...
2,1,0.980031,کیفیت صدای خوبی داره باکس شارژش هم خیلی کاربرد...
3,2,0.798670,جنس بدنه ضعیف یک کالا با دو قیمت متفاوت با همی...
4,1,0.968280,ظاهر جذاب و زیبا در عین حال ساده کیفیت صدا واق...


In [28]:
diff_map = dict()
for (i, row) in hooshvare_df[hooshvare_df['score'] < 0.75].iterrows():
    ollama_label = ollama_df.iloc[i]['label']# type: ignore
    hooshvare_label = row['label']
    if ollama_label != hooshvare_label:
        text = row['text']
        key = str(i) + "_" + str(hooshvare_label) + "_" + str(ollama_label)
        diff_map[key] = (text, row["score"], ollama_df.iloc[i]['reason']) # type: ignore

In [ ]:
filename = "./data/hard_comments_for_claude_labeling.txt"

claude_data = []

for key, value in diff_map.items():
    [comment_id, hooshvare_label, ollama_label] = key.split('_')
    (text, hooshvare_score, ollama_reason) = value
    
    label_map = {0: 'Negative', 1: 'Positive', 2: 'Neutral'}
    
    hooshvare_text = label_map.get(int(hooshvare_label), hooshvare_label)
    ollama_text = label_map.get(int(ollama_label), ollama_label)
    
    comment_data = {
        "id": comment_id,
        "text": text,
        "hooshvare_label": hooshvare_text,
        "hooshvare_score": round(float(hooshvare_score), 3),
        "ollama_label": ollama_text,
        "ollama_reason": ollama_reason
    }
    
    claude_data.append(comment_data)

claude_data.sort(key=lambda x: int(x['id']))

with open(filename, 'w', encoding='utf-8') as f:
    f.write("PERSIAN E-COMMERCE COMMENTS FOR SENTIMENT LABELING\n")
    f.write("=" * 60 + "\n\n")
    f.write("INSTRUCTIONS:\n")
    f.write("Please label each comment with:\n")
    f.write("- 0 for Negative sentiment\n")
    f.write("- 1 for Positive sentiment\n")
    f.write("- 2 for Neutral sentiment\n\n")
    f.write("Consider the HOOSHVARE (BERT) label as more reliable than Ollama label. But your judgement is most important.\n")
    f.write("Provide a brief reason for your decision.\n\n")
    f.write("EXPECTED OUTPUT FORMAT:\n")
    f.write("ID,LABEL,REASON\n")
    f.write("example_id,0,\"Customer complains about product quality\"\n\n")
    f.write("=" * 60 + "\n")
    f.write("COMMENTS TO LABEL:\n")
    f.write("=" * 60 + "\n\n")
    
    for i, comment in enumerate(claude_data, 1):
        f.write(f"COMMENT {i}/{len(claude_data)}\n")
        f.write(f"ID: {comment['id']}\n")
        f.write(f"TEXT: {comment['text']}\n")
        f.write(f"HOOSHVARE (BERT) PREDICTION: {comment['hooshvare_label']} (Score: {comment['hooshvare_score']})\n")
        f.write(f"OLLAMA PREDICTION: {comment['ollama_label']}\n")
        f.write(f"OLLAMA REASON: {comment['ollama_reason']}\n")
        f.write("-" * 40 + "\n\n")

print(f"Created input file for Claude: {filename}")
print(f"Total comments to label: {len(claude_data)}")

Created input file for Claude: hard_comments_for_claude_labeling.txt
Total comments to label: 304


In [ ]:
claude_labels = pd.read_csv('./data/claude_label_for_hard_comments.csv', index_col=None)
claude_labels.head()

,id,label,reason
0,7,0,Customer complains about incorrect wireless ra...
1,43,2,Mixed sentiment: device quality is excellent b...
2,46,2,Mixed review: good packaging and appearance bu...
3,52,1,Neutral: Factual Information
4,53,0,Quality acknowledged but magnetic charger conn...


In [ ]:
final_df = hooshvare_df.copy(deep=True)
for _, row in claude_labels.iterrows():
    final_df.loc[row['id'], "label"] = row["label"]

final_df.drop(columns=['score'], inplace=True)
final_df.to_csv('./data/incredible_offers_product_comments_finalized_labels.csv', index=False)